## Study Vin

Ce notebook à pour objectif de pouvoir étudier un vin en particulier.

Des étapes seront ajouter a chaque fois que l'on cherche quelque chose en particulier sur le vin pour les avoir dans le futurs sur les autres vin.

## Get data

In [ ]:
from transform.processed_tss.ProcessedTimeSeries import ProcessedTimeSeries
from core.pandas_utils import *
from transform.processed_results.main import * 

In [ ]:
import pandas as pd

In [ ]:
tss = ProcessedTimeSeries('tesla', force_update=True)

## Selection du vin 

In [ ]:
ts_vin = tss[tss['vin']=='5YJ3E7EB1KF334219']

In [ ]:
ts_vin['net_capacity'] = 79.0

## Result

In [ ]:
result = (ts_vin.groupby(["vin", "trimmed_in_charge_idx"], observed=True, as_index=False)
        .agg(
            energy_added_min=pd.NamedAgg("charge_energy_added", "min"),
            energy_added_end=pd.NamedAgg("charge_energy_added", "last"),
            soc_diff=pd.NamedAgg("soc", series_start_end_diff),
            inside_temp=pd.NamedAgg("inside_temp", "mean"),
            net_capacity=pd.NamedAgg("net_capacity", "first"),
            range=pd.NamedAgg("range", "first"),
            odometer=pd.NamedAgg("odometer", "first"),
            version=pd.NamedAgg("version", "first"),
            size=pd.NamedAgg("soc", "size"),
            model=pd.NamedAgg("model", "first"),
            date=pd.NamedAgg("date", "first"),
            charging_power=pd.NamedAgg("charging_power", "median"),
            tesla_code=pd.NamedAgg("tesla_code", "first"),
        )
        .eval("energy_added = energy_added_end - energy_added_min")
        .eval("soh = energy_added / (soc_diff / 100.0 * net_capacity)")
        .sort_values(["tesla_code", "vin", "date"])
    )

## Process Result

In [ ]:
def get_processed_results(brand:str) -> DF:
    logger.info(f"{'Processing ' + brand + ' results.':=^{50}}")
    results =  (
        result
        # Some raw estimations may have inf values, this will make mask_out_outliers_by_interquartile_range and force_monotonic_decrease fail
        # So we replace them by NaNs.
        .assign(soh=lambda df: df["soh"].replace([np.inf, -np.inf], np.nan))
        .sort_values(["vin", "date"])
        .pipe(make_charge_levels_presentable)
        .eval(SOH_FILTER_EVAL_STRINGS[brand])
        .pipe(agg_results_by_update_frequency)
        .groupby('vin', observed=True)
        .apply(make_soh_presentable_per_vehicle, include_groups=False)
        .reset_index(level=0)
        .pipe(filter_results_by_lines_bounds, VALID_SOH_POINTS_LINE_BOUNDS, logger=logger)
        .sort_values(["vin", "date"])
    )
    results["soh"] = results.groupby("vin", observed=True)["soh"].ffill()
    results["soh"] = results.groupby("vin", observed=True)["soh"].bfill()
    results["odometer"] = results.groupby("vin", observed=True)["odometer"].ffill()
    results["odometer"] = results.groupby("vin", observed=True)["odometer"].bfill()
    return results

In [ ]:
results = get_processed_results('tesla')

## Resultats

### Validation des valeurs SoH trouvé

In [ ]:
results